# Twitter API and Tweet Collection

References:
- https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/get-and-use-twitter-data-in-python/

In [1]:
# !pip install tweepy

^C


In [2]:
import tweepy
import pandas as pd
import json

In [3]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [4]:
# Set access info to credentials
consumer_key= creds['CONSUMER_KEY']
consumer_secret= creds['CONSUMER_SECRET']
access_token= creds['ACCESS_TOKEN']
access_token_secret= creds['ACCESS_SECRET']

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [245]:
search_words = "'flood' -filter:retweets"
date_since = "2018-09-13"
result_type = "mixed"
# since_id = "1154109669021036546"       # enter the id_str from the last ID of the tweets you got from the last loop

In [246]:
tweets = tweepy.Cursor(api.search,
                       q=search_words,
                       lang="en",
                       since=date_since,
                       result_type=result_type,
#                        since_id=since_id
                      ).items(500)
tweets

In [247]:
tweets_json = [[
                tweet.user.screen_name, 
                tweet.id_str, 
                tweet.created_at, 
                tweet.coordinates, 
                tweet.place,
                tweet.text, 
                tweet.geo 
                ] for tweet in tweets]

In [248]:
tweets_df = pd.DataFrame(tweets_json, columns = [ 
                                                'screen_name', 
                                                'id_str', 
                                                'created_at',
                                                'coordinates',
                                                'place', 
                                                'text',
                                                'geo_location'
                                               ])

In [249]:
tweets_df.tail()

,screen_name,id_str,created_at,coordinates,place,text,geo_location
495,leahmcelrath,1153485320878534656,2019-07-23 02:01:45,None,None,Seriously:\n\nUrban “flood water” isn’t just w...,None
496,JaTapps,1153485316654682112,2019-07-23 02:01:44,None,None,@wolfgar77 @TJimjones @RlalbrechtTroy @Canna42...,None
497,SourD420_,1153485308941479936,2019-07-23 02:01:43,None,None,"So in 24hrs I've had to deal with a blackout, ...",None
498,KVOAWeather,1153485263349407744,2019-07-23 02:01:32,None,None,"TWC cancels Flash Flood Warning for Pima, Sant...",None
499,iembot_twc,1153485263345205250,2019-07-23 02:01:32,None,None,"TWC cancels Flash Flood Warning for Pima, Sant...",None


In [250]:
tweets_df["text_count"] = [len(tweets_df['text'][s].split()) for s in range(len(tweets_df['text']))]

In [251]:
tweets_df.shape

(500, 8)

In [252]:
# Save the tweets dataframe to a csv file
tweets_df.to_csv('../datasets/tweets_flood_1.csv', index=False)